## Chemical kinetics
In chemistry one is often interested in how fast a chemical process proceeds. Chemical reactions (when views as single events on a molecular scale) are probabilitic. However, most reactive systems of interest involve very large number of molecules (a few grams of a simple substance containts on the order of $10^{23}$ molecules. The sheer number allows us to describe this inherently stochastic process deterministically. For example, the following reaction:

$$
2 NO + Br_2 \leftrightarrow 2 NOBr
$$

which describes the equilibrium between nitrogen monoxide (NO) and bromine ($Br_2$) and nitrosyl bromide (NOBr), can be expressed as a set of two reactions (**f**orward and **b**ackward):

$$
2 NO + Br_2 \overset{k_f}{\rightarrow} 2 NOBr \\ 
2 NOBr \overset{k_b}{\rightarrow} 2 NO + Br_2
$$

the rate of the first process (forward) is proportional to the concentration $Br_2$ and the square of the concentration of NO (consider what happends to the collision frequency when the concentration increases). The rate of the second reaction (the backward process) is in analogy proportional to the square of the concentration of NOBr. Using the proportionality constants $k_f$ and $k_b$ we can formulate our system of nonlinear ordinary differential equations as follows:

$$
\frac{dc_1}{dt} = 2(k_b c_3^2 - k_f c_2 c_1^2) \\
\frac{dc_2}{dt} = k_b c_3^2 - k_f c_2 c_1^2 \\
\frac{dc_3}{dt} = 2(k_f c_2 c_1^2 - k_b c_3^2)
$$

where we have denoted the concentration of $[NO],\ [Br_2],\ [NOBr]$ with $c_1,\ c_2,\ c_3$ respectively.

Formulating the system of ODEs this way from the basic elementary reactions is in accordancec with the [law of mass action](https://en.wikipedia.org/wiki/Law_of_mass_action), which can be written as follows:

$$
\frac{dc_i}{dt} = \sum_j S_{ij} r_j \\
r_j = k_j\prod_l c_l^{R_{jl}}
$$

where S is a matrix with the overall net stoichiometric coefficients (positive for net production, negative for net consumption), and R is a matrix with the multiplicities of each reactant for each equation. For our above example:

$$
S = \begin{bmatrix}
-2 & 2 \\
-1 & 1 \\
2 & -2
\end{bmatrix}
$$

$$
R = \begin{bmatrix}
2 & 1 & 0 \\
0 & 0 & 2 
\end{bmatrix}
$$

As a first step we will integrate this system of ordinary differential equations numerically as an initial value problem using the solvers provided in ``scipy``.

In [ ]:
import numpy as np
from scipy.integrate import odeint

By looking at the [documentation](https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.integrate.odeint.html) of odeint we see that we need to provide a callback which computes a vector of derivatives ($\dot{\mathbf{y}} = [\frac{dy_1}{dt}, \frac{dy_2}{dt}, \frac{dy_3}{dt}]$). The expected signature of this callback is:

    f(y: array[float64], t: float64, *args: arbitrary constants) -> dydt: array[float64]
    
in our case we can write it as:

In [ ]:
def rhs(y, t, kf, kb):
    rf = kf * y[0]**2 * y[1]
    rb = kb * y[2]**2
    return [2*(rb - rf), rb - rf, 2*(rf - rb)]

In [ ]:
%load_ext scipy2017codegen.exercise

*Use either the *``%exercise``* or *``%load``* magic to get the exercise / solution respecitvely:*

In [ ]:
# %load ../scipy2017codegen/25_odeint.py
tout = np.linspace(0, 10)
k_vals = 0.42, 0.17  # arbitrary in this case
y0 = [1, 1, 0]
yout = odeint(rhs, y0, tout, k_vals)  # EXERCISE: rhs, y0, tout, k_vals


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(tout, yout)
_ = plt.legend(['NO', 'Br$_2$', 'NOBr'])

Writing the ``rhs`` function by hand for larger reaction systems quickly becomes tedious. Ideally we would like to construct it programatically from some serialized data format describing the set of reactions.

But at the same time, we need the ``rhs`` functions to be fast. Which means that we can't deserialize it for every function call.

So in order to fulfill these two opposing goals we will consider *code generation*. That is: we will write a Python function which deserializes our set of reaction data, and returns a *callback*.

In SymPy there is such a function―``lambdify``―it takes a symbolic expressions and returns a callback in analogy with rhs. In a later notebook, we will show how we can write a deserialization function using SymPy to construct symbolic expressions for a domain specific format, for now we will just use ``rhs`` which we've already written:

In [ ]:
import sympy as sym
sym.init_printing()

In [ ]:
y, k = sym.symbols('y:3'), sym.symbols('kf kb')
ydot = rhs(y, None, *k)
ydot

Now assume that we had constructed ``ydot`` above from some deserialization instead of hard-coding the rate expressions in rhs. Then we could have created a callback corresponding to ``rhs`` using ``lambdify``:

*Use either the *``%exercise``* or *``%load``* magic to get the exercise / solution respecitvely:*

In [ ]:
# %load ../scipy2017codegen/25_lambdify.py
t = sym.symbols('t')  # not used in this case.
f = sym.lambdify((y, t) + k, ydot)  # EXERCISE: (y, t) + k


In [ ]:
plt.plot(tout, odeint(f, y0, tout, k_vals))
_ = plt.legend(['NO', 'Br$_2$', 'NOBr'])

In this example we did not gain much from expressing our problem symbolically and then generate our callback. However, it is quite common that the numerical solver will need a callback calculating the [Jacobian](https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant) of $\dot{\mathbf{y}}$ (given as Dfun). Writing that by hand is both tedious and error prone. Using sympy it is as easy as:

In [ ]:
sym.Matrix(ydot).jacobian(y)

In the next notebook we will look at an example where providing this callback is beneficial for performance. And in the later notebook we will show how the performance of the callback can be optimized.